<a href="https://colab.research.google.com/github/shinichiromizuno/QueryMultiTopic/blob/master/MMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount your Google Drive.
# Connect to GPU in Google Colab.

In [ ]:
################ Preprocess ################

In [ ]:
# Configure working directories.　We use the preprocessed dataset output of Multi-BERTSum by configuring it as the source directory.
work_dir =  '/content/drive/MyDrive/work_MMR'
source_dir = '/content/drive/MyDrive/work_Multi_BERTSum'

In [ ]:
# Skipping Step1~4, as we use the preprocessed dataset output of Multi-BERTSum.

In [ ]:
# Preprocesing Dataset
import json
import torch
import os
import random
from glob import glob
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

src_doc_train = []
src_doc_valid = []
tgt_list_valid = []
src_doc_test = []
tgt_list_test = []
for data in sorted(glob(source_dir + '/json_data_labeled/*')):
  filetype = data.split('/')[-1].split('.')[1]
  json_open = open(data)
  json_load = json.load(json_open)
  src_txt = json_load['src_txt']
  tagged_txt = [nltk.pos_tag(nltk.word_tokenize(sentence)) for sentence in src_txt]
  selected_txt = []
  for tagged_sent in tagged_txt:
    sent_after_selection = []
    for word_tupple in tagged_sent:
      if word_tupple[1] in ['JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        sent_after_selection.append(word_tupple[0]) 
    selected_txt.append(' '.join(sent_after_selection))
  if filetype == 'train':
    src_doc_train.append(selected_txt)
  elif filetype == 'valid':
    src_doc_valid.append(selected_txt)
    tgt_list = []
    for i in range(1,18):
      tgt_list.append(json_load['tgt'+str(i)])
    tgt_list_valid.append(tgt_list)
  elif filetype == 'test':
    src_doc_test.append(selected_txt)
    tgt_list = []
    for i in range(1,18):
      tgt_list.append(json_load['tgt'+str(i)])
    tgt_list_test.append(tgt_list)

src_doc_train_concat = []
for doc in src_doc_train:
  src_doc_train_concat.append(' '.join(doc))

vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
transformer = TfidfTransformer()

tf = vectorizer.fit_transform(src_doc_train_concat)
tfidf = transformer.fit_transform(tf)

sdgsdir = '/content/drive/MyDrive/DatasetESG/0-SDGs/'

all_query_txt = []
for goal in sorted(glob(sdgsdir + '/*')):
  query_txt = []
  with open(goal, mode='r') as f:
    for line in f:
      query_txt.append(line.strip())
  tagged_query_txt = nltk.pos_tag(nltk.word_tokenize(' '.join(query_txt)))
  query_txt_after_selection = []
  for word_tupple in tagged_query_txt:
    if word_tupple[1] in ['JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
      query_txt_after_selection.append(word_tupple[0])
  all_query_txt.append(' '.join(query_txt_after_selection))

query_src_cs_list_valid = []
src_src_cs_list_valid = []
for src_doc in src_doc_valid:
  tf_src_doc = vectorizer.transform(src_doc)
  tfidf_src_doc = transformer.transform(tf_src_doc)
  tf_all_query_txt = vectorizer.transform(all_query_txt)
  tfidf_all_query_txt = transformer.transform(tf_all_query_txt)
  cs = cosine_similarity(tfidf_all_query_txt, tfidf_src_doc)
  query_src_cs_list_valid.append(cs)
  cs = cosine_similarity(tfidf_src_doc, tfidf_src_doc)
  src_src_cs_list_valid.append(cs)

query_src_cs_list_test = []
src_src_cs_list_test = []
for src_doc in src_doc_test:
  tf_src_doc = vectorizer.transform(src_doc)
  tfidf_src_doc = transformer.transform(tf_src_doc)
  tf_all_query_txt = vectorizer.transform(all_query_txt)
  tfidf_all_query_txt = transformer.transform(tf_all_query_txt)
  cs = cosine_similarity(tfidf_all_query_txt, tfidf_src_doc)
  query_src_cs_list_test.append(cs)
  cs = cosine_similarity(tfidf_src_doc, tfidf_src_doc)
  src_src_cs_list_test.append(cs)

dataset = {'query_src_cs_list_valid': query_src_cs_list_valid, 'src_src_cs_list_valid': src_src_cs_list_valid, 'tgt_list_valid': tgt_list_valid, 'query_src_cs_list_test': query_src_cs_list_test, 'src_src_cs_list_test': src_src_cs_list_test, 'tgt_list_test': tgt_list_test}
torch.save(dataset, work_dir + '/dataset.pt')  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
################ Validation ################

In [ ]:
import numpy as np

def mmr(query_src_cs_list, src_src_cs_list, lambda_):
  all_sumset = []
  for d in range(len(query_src_cs_list)):
    qry = np.array(query_src_cs_list[d])
    src = np.array(src_src_cs_list[d])
    doc_sumset = []
    for i in range(len(qry)):
      sumset = []
      diff = lambda_ * qry[i] - (1-lambda_) * src
      for j in range(len(diff)):
        diff[j, j] = -float('inf')
      qry_max = np.argmax(qry[i])
      sumset.append(qry_max)
      diff[:, qry_max] -= float('inf')
      for j in range(len(diff) - 1):
        ind = np.zeros(len(diff), dtype=bool)
        ind[sumset] = True
        maxind = np.unravel_index(np.argmax(diff[ind]), diff.shape)[1]
        sumset.append(maxind)
        diff[:, maxind] -= float('inf')
      doc_sumset.append(sumset)
    all_sumset.append(doc_sumset)
  return all_sumset

In [ ]:
def predict_goals(all_sumset, num_sent):
  mmr_pred = []
  for sumset in all_sumset:
    ind = np.zeros(len(sumset))
    ind[sumset[0:num_sent]] = 1
    mmr_pred.append(ind.tolist())
  return mmr_pred

In [ ]:
# Showing F1 Score and Accuracy in all an by qry
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def show_statistics_valid(all_predited_list, all_tgt_list, thres_list):
  best_all_predited_list = []
  for i in range(17):
    by_thres_predicted = []
    f1_list, acc_list = [], []
    for j, thres in enumerate(thres_list):
      by_qry_predicted = []
      by_qry_tgt = []
      for doc_pred, doc_tgt in zip(all_predited_list, all_tgt_list):
        by_qry_predicted += doc_pred[j][i]
        by_qry_tgt += doc_tgt[i]
      f1 = f1_score(by_qry_predicted, by_qry_tgt)
      acc = accuracy_score(by_qry_predicted, by_qry_tgt)
      f1_list.append(f1)
      acc_list.append(acc)
    bs = np.argmax(f1_list)
    print(f'Goal{i+1}, threshold: {thres_list[bs]}, f1_score: {round(f1_list[bs], 3)}, accuracy: {round(acc_list[bs], 3)}')
    best_all_predited_list.append([doc_pred[bs][i] for doc_pred in all_predited_list])
  
  best_all_predited_list_t = np.array(best_all_predited_list).T.tolist()
  single_predicted = []
  single_tgt = []
  for doc_pred, doc_tgt in zip(best_all_predited_list_t, all_tgt_list):
    for qry_pred, qry_tgt in zip(doc_pred, doc_tgt):
      single_predicted += qry_pred
      single_tgt += qry_tgt
  best_f1 = f1_score(single_predicted, single_tgt)
  best_acc = accuracy_score(single_predicted, single_tgt)
  print(f'Total Best f1_score:{round(best_f1, 3)}, accuracy:{round(best_acc, 3)}')

In [ ]:
import torch
from glob import glob
import numpy as np

def validate_execute(validation_list):
  dataset = torch.load(glob(work_dir + '/dataset.pt')[0])
  for lambda_ in validation_list:
    print(lambda_)
    all_sumset = mmr(query_src_cs_list_valid, src_src_cs_list_valid, lambda_=lambda_)
    all_sumset_t = np.array(all_sumset).T
    num_sent_range = range(5, 305, 5)
    src_pred_list = []
    for i in range(17):
      src_pred_list_by_thres = []
      for j in num_sent_range:
        src_pred = predict_goals(all_sumset_t[i], num_sent=j)
        src_pred_list_by_thres.append(src_pred)
      src_pred_list.append(src_pred_list_by_thres)
    src_pred_list_t = np.array(src_pred_list).T.tolist()
    show_statistics_valid(src_pred_list_t, dataset['tgt_list_valid'], [i for i in num_sent_range])

In [ ]:
# Showing Optimal Hyperparameters
valiation_list = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0]
validate_execute(valiation_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, threshold: 285, f1_score: 0.031, accuracy: 0.59
Goal2, threshold: 20, f1_score: 0.04, accuracy: 0.963
Goal3, threshold: 120, f1_score: 0.154, accuracy: 0.802
Goal4, threshold: 115, f1_score: 0.088, accuracy: 0.827
Goal5, threshold: 30, f1_score: 0.135, accuracy: 0.929
Goal6, threshold: 10, f1_score: 0.069, accuracy: 0.977
Goal7, threshold: 80, f1_score: 0.141, accuracy: 0.86
Goal8, threshold: 185, f1_score: 0.156, accuracy: 0.716
Goal9, threshold: 300, f1_score: 0.126, accuracy: 0.572
Goal10, threshold: 110, f1_score: 0.057, accuracy: 0.829
Goal11, threshold: 160, f1_score: 0.093, accuracy: 0.758
Goal12, threshold: 185, f1_score: 0.153, accuracy: 0.723
Goal13, threshold: 235, f1_score: 0.115, accuracy: 0.656
Goal14, threshold: 20, f1_score: 0.074, accuracy: 0.952
Goal15, threshold: 50, f1_score: 0.089, accuracy: 0.905
Goal16, threshold: 75, f1_score: 0.079, accuracy: 0.877
Goal17, threshold: 295, f1_score: 0.103, accuracy: 0.57


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Total Best f1_score:0.107, accuracy:0.794


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


...

Total Best f1_score:0.114, accuracy:0.812


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, threshold: 245, f1_score: 0.039, accuracy: 0.649
Goal2, threshold: 90, f1_score: 0.046, accuracy: 0.867
Goal3, threshold: 245, f1_score: 0.126, accuracy: 0.638
Goal4, threshold: 135, f1_score: 0.065, accuracy: 0.796
Goal5, threshold: 230, f1_score: 0.074, accuracy: 0.657
Goal6, threshold: 60, f1_score: 0.028, accuracy: 0.906
Goal7, threshold: 250, f1_score: 0.101, accuracy: 0.633
Goal8, threshold: 260, f1_score: 0.125, accuracy: 0.611
Goal9, threshold: 245, f1_score: 0.122, accuracy: 0.639
Goal10, threshold: 300, f1_score: 0.042, accuracy: 0.564
Goal11, threshold: 245, f1_score: 0.1, accuracy: 0.649
Goal12, threshold: 270, f1_score: 0.119, accuracy: 0.604
Goal13, threshold: 245, f1_score: 0.104, accuracy: 0.638
Goal14, threshold: 165, f1_score: 0.061, accuracy: 0.755
Goal15, threshold: 105, f1_score: 0.075, accuracy: 0.83
Goal16, threshold: 195, f1_score: 0.046, accuracy: 0.708
Goal17, threshold: 300, f1_score: 0.078, accuracy: 0.552


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Total Best f1_score:0.086, accuracy:0.688


In [ ]:
################ Test ################

In [ ]:
# Showing F1 Score and Accuracy in all an by qry
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def show_statistics_test(all_predited_list, all_tgt_list):
  by_qry_tgt_list = []
  by_qry_pred_list = []
  for i in range(17):
    by_qry_tgt = []
    for data in all_tgt_list:
      by_qry_tgt += data[i]
    by_qry_pred = []
    for data in all_predited_list:
      by_qry_pred += data[i]
    f1 = f1_score(by_qry_pred, by_qry_tgt)
    acc = accuracy_score(by_qry_pred, by_qry_tgt)
    print(f'Goal{i+1}, f1_score: {round(f1, 3)}, accuracy: {round(acc, 3)}')

    by_qry_tgt_list.append(by_qry_tgt)
    by_qry_pred_list.append(by_qry_pred)
  
  tgt_list_all = []
  pred_list_all = []
  for by_qry_tgt in by_qry_tgt_list:
    tgt_list_all += by_qry_tgt
  for by_qry_pred in by_qry_pred_list:
    pred_list_all += by_qry_pred
  
  f1_all = f1_score(pred_list_all, tgt_list_all)
  acc_all = accuracy_score(pred_list_all, tgt_list_all)
  print(f'Total, f1_score: {round(f1_all, 3)}, accuracy: {round(acc_all, 3)}')

In [ ]:
import torch
from glob import glob
import numpy as np

def test_execute(sent_length_list, lambda_):
  dataset = torch.load(glob(work_dir + '/dataset.pt')[0])
  print(lambda_)
  all_sumset = mmr(query_src_cs_list_test, src_src_cs_list_test, lambda_=lambda_)
  all_sumset_t = np.array(all_sumset).T 
  
  src_pred_list = []
  for i in range(17):
    src_pred = predict_goals(all_sumset_t[i], num_sent=sent_length_list[i])
    src_pred_list.append(src_pred)
  src_pred_list_t = np.array(src_pred_list).T.tolist()
  show_statistics_test(src_pred_list_t, dataset['tgt_list_test'])

In [ ]:
# Showing Test Results
sent_length_list = [50, 20, 120, 115, 30, 10, 80, 185, 295, 110, 160, 185, 200, 20, 50, 75, 295]
all_sumset_t = test_execute(sent_length_list, 0.9)

0.9


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, f1_score: 0.017, accuracy: 0.92
Goal2, f1_score: 0.054, accuracy: 0.965
Goal3, f1_score: 0.087, accuracy: 0.8
Goal4, f1_score: 0.075, accuracy: 0.825
Goal5, f1_score: 0.115, accuracy: 0.936
Goal6, f1_score: 0.125, accuracy: 0.972
Goal7, f1_score: 0.113, accuracy: 0.861
Goal8, f1_score: 0.131, accuracy: 0.714
Goal9, f1_score: 0.097, accuracy: 0.563
Goal10, f1_score: 0.041, accuracy: 0.825
Goal11, f1_score: 0.072, accuracy: 0.751
Goal12, f1_score: 0.125, accuracy: 0.715
Goal13, f1_score: 0.116, accuracy: 0.684
Goal14, f1_score: 0.079, accuracy: 0.954
Goal15, f1_score: 0.095, accuracy: 0.913
Goal16, f1_score: 0.045, accuracy: 0.875
Goal17, f1_score: 0.084, accuracy: 0.567
Total, f1_score: 0.093, accuracy: 0.814
